In [15]:
from data_retrieval import alpaca
from data_retrieval import crypto_market
from data_retrieval import getbinance
from data_retrieval import sp500_index

from functions_graphs import functions
from user_input import risk_assesment

import pandas as pd
import hvplot.pandas
import numpy

In [2]:
# user_input = ra.get_user_risk_tolerance()

In [39]:
def get_crypto_dict():
    # Returns a dictionary with ticker as index and its dataframe as a value
    cryptos = ['ADA', 'BNB', 'BTC', 'DOT', 'ETH', 'LINK', 'LTC', 'VET', 'XLM', 'XRP']
    crypto_dict = {}
    for ticker in cryptos:
        df = getbinance.Binance(ticker, 3)
        df = pd.DataFrame(df)
        crypto_dict[ticker] = df

    return crypto_dict

In [40]:
dictionary = get_crypto_dict()

In [41]:
def get_std(crypto_dfs):
    # Returns a dictionary of all crypto stds
    cryptos = ['ADA', 'BNB', 'BTC', 'DOT', 'ETH', 'LINK', 'LTC', 'VET', 'XLM', 'XRP']
    crypto_std = {}
    for ticker in cryptos:
        df = crypto_dfs[ticker]
        c_name = f"{ticker}USDT_Close"
        funct = functions.Functions(df, c_name)
        df = funct.daily_returns()
        df.dropna(inplace=True)

        crypto_std[ticker] = funct.standard_deviation()
#         display(df.head())
    return crypto_std

In [47]:
crypto_std = get_std(dictionary)

In [53]:
columns = ['standard deviation']

In [54]:
crypto_std_df = pd.DataFrame(crypto_std, index=columns)

In [68]:
crypto_std_df = crypto_std_df.T
crypto_std_df

,standard deviation
ADA,0.060137
BNB,0.059834
BTC,0.039431
DOT,0.083545
ETH,0.052380
LINK,0.071551
LTC,0.055060
VET,0.068487
XLM,0.063335
XRP,0.064917


In [70]:
crypto_std_df = crypto_std_df.sort_values(by=['standard deviation'])

In [71]:
crypto_std_df.hvplot.bar(title='Standard Deviations of 10 Crypto')

:Bars   [index]   (standard deviation)

In [38]:
# Make a dictionary, save the main key as the ticker, in the values save the std and the percent change series in nexted dictionaries

In [8]:
# 0.0 -.057 Low
# .057 - .073 Med
# .073 - 1 High
def sort_crypto_std(crypto_std):
    # Returns a dictionary with all crypto std's sorted from low to high risk
    high_risk = {}
    med_risk = {}
    low_risk = {}
    for ticker, std in crypto_std.items():
        if float(std) <= 0.057:
            low_risk[ticker] = std
        if float(std) > 0.057 and float(std) <= 0.073:
            med_risk[ticker] = std
        if float(std) > 0.073:
            high_risk[ticker] = std


    return {"low":low_risk, "med":med_risk, "high":high_risk}

In [9]:
sorted_std = sort_crypto_std(crypto_std)
for t,v in sorted_std['low'].items():
    print(f"Low Risk Cryptos - {t}:{v}")
print("----------")
for t,v in sorted_std['med'].items():
    print(f"Medium Risk Cryptos - {t}:{v}")  
print("----------")
for t,v in sorted_std['high'].items():
    print(f"High Risk Cryptos - {t}:{v}")

Low Risk Cryptos - BTC:0.03938787125238744
Low Risk Cryptos - ETH:0.05235430550261196
Low Risk Cryptos - LTC:0.055032408494728645
----------
Medium Risk Cryptos - ADA:0.05993643980158168
Medium Risk Cryptos - BNB:0.05977611996900103
Medium Risk Cryptos - LINK:0.07157934652302073
Medium Risk Cryptos - VET:0.06848841123396819
Medium Risk Cryptos - XLM:0.06329186121298729
Medium Risk Cryptos - XRP:0.06486879336788266
----------
High Risk Cryptos - DOT:0.08360492871492359


In [20]:
cmc = crypto_market.cmc200(1)

In [16]:
sp500 = sp500_index.sp500(1)

In [17]:
sp500.columns =['SP500-close']
sp500

,SP500-close
Date,
2020-08-20,3385.510010
2020-08-21,3397.159912
2020-08-24,3431.280029
2020-08-25,3443.620117
2020-08-26,3478.729980
...,...
2021-08-13,4468.000000
2021-08-16,4472.939941
2021-08-17,4448.080078


In [21]:
cmc.columns = ['CMC_close']
cmc

,CMC_close
Date,
2020-08-20,262.244995
2020-08-21,253.847000
2020-08-24,259.174011
2020-08-25,247.363007
2020-08-26,251.794998
...,...
2021-08-13,1188.456055
2021-08-16,1169.593994
2021-08-17,1118.203979


In [22]:
index = pd.concat([sp500, cmc], axis=1)
index.dropna(inplace=True)
index

,SP500-close,CMC_close
Date,,
2020-08-20,3385.510010,262.244995
2020-08-21,3397.159912,253.847000
2020-08-24,3431.280029,259.174011
2020-08-25,3443.620117,247.363007
2020-08-26,3478.729980,251.794998
...,...,...
2021-08-13,4468.000000,1188.456055
2021-08-16,4472.939941,1169.593994
2021-08-17,4448.080078,1118.203979


In [31]:
# plot_cmc = cmc.hvplot.line(title='CMC200 Index', rot=90)

In [32]:
# plot_snp = snp_index.hvplot.line(title='SNP 500 Index', rot=90)

In [38]:
# overlay_index = plot_cmc * plot_snp
# overlay_index.opts(
#     height=500,
#     width=1000,
#     xlabel='Return',
#     ylabel='Date',
#     title='Index Overlayed')

In [34]:
plot = index.hvplot.line(title='CMC200 and S&P500 Overlayed')

In [35]:
hvplot.show(plot)

Launching server at http://localhost:60099


In [27]:
sp_cmc = index.rolling(window=30).mean().dropna()
sp_cmc

,SP500-close,CMC_close
Date,,
2020-10-02,3387.842676,235.243468
2020-10-05,3388.612345,233.859968
2020-10-06,3387.406014,232.599635
2020-10-07,3387.011344,231.208935
2020-10-08,3387.118343,230.384667
...,...,...
2021-08-13,4387.898649,904.615306
2021-08-16,4391.918652,916.370140
2021-08-17,4395.403320,925.206038


In [28]:
sp_cmc.hvplot.line(title='CMC200 and S&P500 Overlayed')

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [33]:
sp_cmc_avg = index.mean(axis=1)
sp_cmc_avg_30 = sp_cmc_avg.rolling(window=30).mean()
sp_cmc_avg_30.hvplot.line(title='CMC200 and S&P500 Averaged')

:Curve   [Date]   (0)